<a href="https://colab.research.google.com/github/eliaslimmer/PraktikumMushroom/blob/main/data_creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Requirements Installation



In [ ]:
##test

In [ ]:
!pip install transformers datasets evaluate accelerate
!pip install --upgrade datasets
!pip install scipy
!pip install seqeval

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 123.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 99.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [ ]:
from google.colab import drive
import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, DataCollatorForTokenClassification
import evaluate
from datasets import load_dataset
import torch.nn.functional as F
import gc
import json
from tqdm import tqdm
from scipy.stats import spearmanr #For the spearman correlation computation

# Clone from Git


In [ ]:
# Clone the GitHub repository
!git clone https://github.com/eliaslimmer/PraktikumMushroom.git

# Change directory to the cloned repository
%cd PraktikumMushroom

import helpers
!ls

Cloning into 'PraktikumMushroom'...
remote: Enumerating objects: 69, done.
remote: Counting objects: 100% (69/69), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 69 (delta 33), reused 48 (delta 18), pack-reused 0 (from 0)
Receiving objects: 100% (69/69), 4.99 MiB | 18.85 MiB/s, done.
Resolving deltas: 100% (33/33), done.
/content/PraktikumMushroom
data				mushroom.en-val.v2.extra.jsonl
data_creation.ipynb		mushroom.fr-tst.v1.extra.jsonl
helpers.py			mushroom.fr-val.v2.extra.jsonl
mushroom.de-tst.v1.extra.jsonl	outputs_partial_en.jsonl
mushroom.de-val.v2.extra.jsonl	__pycache__
mushroom.en-tst.v1.extra.jsonl	README.md


In [ ]:
!git pull origin main

remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3 (delta 1), reused 3 (delta 1), pack-reused 0 (from 0)
Unpacking objects: 100% (3/3), 155.25 KiB | 5.35 MiB/s, done.
From https://github.com/eliaslimmer/PraktikumMushroom
 * branch            main       -> FETCH_HEAD
   491276d..dc068cf  main       -> origin/main
Updating 491276d..dc068cf
Fast-forward
 en-pred.jsonl | 154 ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
 1 file changed, 154 insertions(+)
 create mode 100644 en-pred.jsonl


#Login to huggingface

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

# 1. Download the MKQA datasets



In [ ]:
lang = 'en' #Set the language. Could be 'de' if german, 'fr' if french
language_data = helpers.prepare_dataset(lang)

#Print the length of the dataset and a sample query and answer
print(f"Length of {lang} dataset: {len(language_data['queries'])}, "
      f"sample {lang} query: {language_data['queries'][0]}, "
      f"sample {lang} answer: {', '.join(language_data['answers'][0])}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

mkqa.py: 0.00B [00:00, ?B/s]

The repository for mkqa contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mkqa.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Length of en dataset: 10000, sample en query: how long did it take the twin towers to be built, sample en answer: 11.0 years


# 2. Generate Answers and probabilities

In [ ]:
#Load the model and tokenizer from Huggingface
model_id = "meta-llama/Llama-3.1-8B-Instruct" #We used the Llama-3.1-8B-Instruct model
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained(model_id, padding_side='left')
tokenizer.pad_token = tokenizer.eos_token

#Generate the answers
file_path = f'outputs_partial_{lang}.jsonl'
helpers.generate_answers(model, tokenizer, language_data['queries'], lang, file_path) #If you want to change the language, change the 'lang' at the code block above.

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Generating: 100%|██████████| 2500/2500 [2:45:46<00:00,  3.98s/it]


#3. Span mapping

In [ ]:
file_path = f'outputs_partial_{lang}.jsonl'
model_id = "meta-llama/Llama-3.1-8B-Instruct" #We used the Llama-3.1-8B-Instruct model
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained(model_id, padding_side='left')
tokenizer.pad_token = tokenizer.eos_token

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [ ]:
span_file_path = f'outputs_partial_span_{lang}.jsonl'
helpers.span_mapping(file_path, span_file_path, tokenizer)

#4. Convert soft labels to hard labels

In [ ]:
input_path = span_file_path
output_path = f'train_dataset_{lang}.jsonl'
helpers.generate_hard_labels(input_path, output_path)
train_data_path = output_path #Set the train_data_path

#5. Validation dataset reformat

In [ ]:
val_input_path = f'mushroom.{lang}-val.v2.extra.jsonl'
val_output_path = f'validation_dataset_{lang}.jsonl'
helpers.validation_reformat(val_input_path, val_output_path)
val_data_path = val_output_path #Set the val_data_path

#6. Model Training

In [ ]:
LABEL_LIST = ['O', 'B']
MODEL_NAME = 'FacebookAI/xlm-roberta-base'
data_files = {
        "train": train_data_path,
        "validation": val_data_path
    }
helpers.train_model(MODEL_NAME, LABEL_LIST, data_files, output_dir=f"./results_xmr_{lang}")

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at FacebookAI/xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.798500,3.516951,0.054571,1.000000,0.103495,0.054571
2,0.422600,4.150593,0.054571,1.000000,0.103495,0.054571
3,0.994000,4.704367,0.054571,1.000000,0.103495,0.054571


Training completed. Checkpoints saved to: ./results_xmr_en


#7. Model Testing

In [ ]:
test_path = f'mushroom.{lang}-tst.v1.extra.jsonl'
helpers.test_model(
    MODEL_NAME,
    test_lang=lang,
    model_path=f'results_xmr_{lang}/checkpoint-15000',
    data_path=test_path
)

Generating test split: 0 examples [00:00, ? examples/s]

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


Labels saved to en-hard_labels.json and en-soft_labels.json
Prediction file saved to en-pred.jsonl


#8. Evaluation

In [ ]:
reference = helpers.load_jsonl(test_path) #Change the filename
hard_references, soft_references = helpers.convert_reference_to_dict(reference)
predictions = helpers.load_jsonl(f"{lang}-pred.jsonl")

results = helpers.evaluate_predictions(predictions, hard_references, soft_references)

print("📊 Evaluation Results:")
print(f"Mean IoU: {results['mean_iou']:.4f}")
print(f"Mean Spearman: {results['mean_spearman']:.4f}")

📊 Evaluation Results:
Mean IoU: 0.4789
Mean Spearman: 0.2181
